<a href="https://colab.research.google.com/github/eva-hod/socialInequality/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math

In [2]:
#В этот блок вынесены методы, которые нужны для расчётов
class citizen:
    def __init__(self, ID, money):
        self.ID = ID
        self.money = money
    def __str__(self):
        return "ID: {}; money: {}".format(self.ID, self.money)

#Совершает розыгрыш между двумя горожанами
def makeDeal(c1, c2):
    bank = 0
    if c1.money > 0:
        bank+=1
        c1.money-=1
    if c2.money > 0:
        bank+=1
        c2.money-=1
    if bank == 0:
        return
    else:
        pr = random.random();
        if (pr > 0.499):
            c1.money+=bank
        else:
            c2.money+=bank

#Случайно выбирает двух жителей из города            
def selectTwoCitizens(listOfCitizens):
    result = np.empty(0)
    if (listOfCitizens.shape[0]>2):
        result = np.random.choice(listOfCitizens, 2, False)
        return result
    elif (listOfCitizens.shape[0]==2):
        result = listOfCitizens
        return result
    else:
        return result

#Делит жителей на пары и проводит в каждой паре розыгрыш
def makeDealsInPairs(listOfCitizens):
    localCopy = np.copy(listOfCitizens)
    for i in range(listOfCitizens.shape[0]//2):
        currentPair = selectTwoCitizens(localCopy)
        if (currentPair.shape[0] == 2):
            makeDeal(currentPair[0], currentPair[1])
            index = np.where(localCopy == currentPair[0])
            localCopy = np.delete(localCopy, index)
            index = np.where(localCopy == currentPair[1])
            localCopy = np.delete(localCopy, index)

#Строит текущее распределение c(t)
def makeDistribution(listOfCitizens, initMoney):
    distribution = np.zeros((listOfCitizens.shape[0]*initMoney))
    for c in listOfCitizens:
        distribution[c.money]+=1
    distribution=(1/listOfCitizens.shape[0])*distribution
    return distribution

#Строит распределение c*(t)
def makeCStar(listOfCitizens, initMoney):
    C = 1 / initMoney
    cstar = np.zeros((listOfCitizens.shape[0]*initMoney))
    for i in range(cstar.shape[0]):
        cstar[i] = C * math.exp(-i/initMoney)
    return cstar

In [3]:
#В этот блок вынесены методы, которые нужны для расчётов
#Считает mixingTime для одного набора N, s с чертой и sigma
def makeOneRealization(citizenNum, initMoney, sigma):
    mixingTime = 0
    #создаю и заселяю город, раздаю всем деньги
    city = np.empty((citizenNum), dtype = object)
    for i in range(city.shape[0]):
        city[i] = citizen(i, initMoney)
    #считаю равновесную кривую
    cs = makeCStar(city, initMoney)
    #Условие остановки расчёта
    stop = False
    #Основной цикл
    while stop == False:
        #Разбиваю всех на пары, совершаю сделку в каждой паре
        makeDealsInPairs(city)
        #Строю текущее распределение
        c = makeDistribution(city, initMoney)
        #Расчёт условия остановки
        #Евклидово расстояние между текущей и равновесной кривой
        diff = c - cs
        left = 0
        for i in range(diff.shape[0]):
            left+=(diff[i])**2
        left = np.sqrt(left)
        #Правая часть формулы
        right = ((math.log(1/sigma))**(1/2))/(citizenNum**(1/2))
        if (left < right):
            stop = True
        else:
            stop = False
        mixingTime+=1
    return mixingTime
#Считает mixingTime для одного значения количества жителей города. 
#Каждую итерацию цикла просит пользователя ввести что-нибудь с клавиатуры
#Заканчивает расчёт mixingTime когда пользователь ввёл символ q
def makeOneRealizationPB(citizenNum, initMoney, sigma):
    mixingTime = 0
    sym = ''
    #создаю и заселяю город, раздаю всем деньги
    city = np.empty((citizenNum), dtype = object)
    for i in range(city.shape[0]):
        city[i] = citizen(i, initMoney)
    #считаю равновесную кривую
    cs = makeCStar(city, initMoney)
    #Условие остановки расчёта
    stop = False
    #Основной цикл
    while sym != 'q':
        #Разбиваю всех на пары, совершаю сделку в каждой паре
        makeDealsInPairs(city)
        #Строю текущее распределение
        c = makeDistribution(city, initMoney)
        #Расчёт условия остановки
        #Евклидово расстояние между текущей и равновесной кривой
        left = math.dist(c,cs)
        #Правая часть формулы
        right = ((math.log(1/sigma))**(1/2))/(citizenNum**(1/2))
        if (left < right):
            stop = True
        else:
            stop = False
        
        mixingTime+=1
        print("||c-cs||L2 "+str(left))
        print("right "+str(right))
        print(mixingTime)
        plt.plot(c)
        plt.plot(cs)
        plt.show()
        sym = input() 

In [ ]:
#Основной блок с расчётом
numDays = 0 #mixingTime
sigma = 0.1 # 0 < sigma < 1 - параметр для проверки достижения равновесной кривой
citizenNum = 10 #N - число жителей
initMoney = 10 #sinit оно же s с чертой

TFromN = [] # Полученная экспериментально зависимость mixingTime от N

#Считает TFromN от 10 до 10000 c шагом в 10 человек
while citizenNum < 10000:
    res = makeOneRealization(citizenNum, initMoney, sigma)
    t = res[0]
    TFromN.append([citizenNum, t])
    print([citizenNum, t])
    citizenNum += 100

[10, 6]
[110, 76]
[210, 56]
[310, 80]
[410, 97]
[510, 108]
[610, 117]
[710, 125]
[810, 146]
[910, 191]
[1010, 184]
[1110, 177]
[1210, 194]
[1310, 209]
[1410, 210]
[1510, 179]
[1610, 259]
[1710, 277]
[1810, 225]
[1910, 242]
[2010, 234]
[2110, 313]
[2210, 342]
[2310, 301]
[2410, 343]
[2510, 323]
[2610, 486]
[2710, 353]
[2810, 304]
[2910, 292]
[3010, 369]
[3110, 360]
[3210, 392]
[3310, 491]
[3410, 441]
[3510, 364]
[3610, 597]
[3710, 406]
[3810, 689]
[3910, 417]
[4010, 493]
[4110, 471]
[4210, 750]
[4310, 528]
[4410, 487]
[4510, 510]
[4610, 636]
[4710, 568]
[4810, 624]
[4910, 545]
[5010, 696]
[5110, 731]
[5210, 656]
[5310, 936]
[5410, 603]


In [ ]:
#Блок с отрисовкой
x = np.zeros(len(TFromN))
y = np.zeros(len(TFromN))
for i in range(len(TFromN)):
    x[i] = TFromN[i][0]
    y[i] = TFromN[i][1]
plt.grid()
plt.title("citizens "+str(x[0])+" - "+str(x[x.shape[0]-1])+", initMoney "+str(initMoney)+", sigma "+str(sigma)+", step 100 citizens")
plt.xlabel("N, citizens")
plt.ylabel("mixingTime, days")
plt.yscale('log')
plt.xscale('log')
plt.plot(x,y, label = 'Экспериментально полученное mixingTime(N)')
plt.legend(loc = 'lower right')
plt.show()